In [12]:
# importing modules
import pandas as pd
import numpy as np
import pandas as pd
import geopandas as gpd
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool
from bokeh.palettes import Viridis256, Inferno256, Magma256
from bokeh.palettes import BrBG, PiYG, RdGy, RdYlGn, YlGnBu
from bokeh.models import LinearColorMapper, ColorBar
#import streamlit as st
# #from streamlit_folium import st_folium
#import folium
#from leaflet import Map, GeoJSON

In [13]:
# Reading the data
rheas_chirps_raw=pd.read_csv('data/chirps/ken_n_25_chirps__newmask_2022_2023_maize.csv')
rheas_tamsat_raw=pd.read_csv('data/tamsat/ken_n_25_tamsat__newmask_2022_2023_maize.csv')

In [14]:
# remove non-agricultural counties
rheas_tamsat_raw=rheas_tamsat_raw.loc[(rheas_tamsat_raw['cname']!='Marsabit')&
                                      (rheas_tamsat_raw['cname']!='Mandera')&
                                      (rheas_tamsat_raw['cname']!='Wajir')&
                                      (rheas_tamsat_raw['cname']!='Lamu')&
                                      (rheas_tamsat_raw['cname']!='Nairobi')&
                                      (rheas_tamsat_raw['cname']!='Mombasa')&
                                      (rheas_tamsat_raw['cname']!='Turkana')&
                                      (rheas_tamsat_raw['cname']!='Mombasa')&
                                      (rheas_tamsat_raw['cname']!='Garissa')]
                                    #  (rheas_tamsat_raw['cname']!='Isiolo')
rheas_chirps_raw=rheas_chirps_raw.loc[(rheas_chirps_raw['cname']!='Marsabit')&
                                      (rheas_chirps_raw['cname']!='Mandera')&
                                      (rheas_chirps_raw['cname']!='Wajir')&
                                      (rheas_chirps_raw['cname']!='Lamu')&
                                      (rheas_chirps_raw['cname']!='Nairobi')&
                                      (rheas_chirps_raw['cname']!='Mombasa')&
                                      (rheas_chirps_raw['cname']!='Turkana')&
                                      (rheas_chirps_raw['cname']!='Mombasa')&
                                      (rheas_chirps_raw['cname']!='Garissa')]
                                    #   (rheas_tamsat_raw['cname']!='Isiolo')

In [15]:
# tamsat change dates to months or years and calculate yield
rheas_tamsat_raw['planting']=pd.to_datetime(rheas_tamsat_raw['planting'], format='ISO8601')
rheas_tamsat_raw['harvest']=pd.to_datetime(rheas_tamsat_raw['harvest'], format='ISO8601')
rheas_tamsat_raw['harvest']=pd.to_datetime(rheas_tamsat_raw['harvest'], format='ISO8601')
rheas_tamsat_raw['year'] = rheas_tamsat_raw['planting'].dt.year
rheas_tamsat_raw['yield']=rheas_tamsat_raw['gwad']/1000
# rheas_tamsat_raw['harvest']=pd.to_datetime(rheas_tamsat_raw['harvest'], format='%Y/%m/%d')

# chirps change dates to months or years and calculate yield
rheas_chirps_raw['planting']=pd.to_datetime(rheas_chirps_raw['planting'], format='ISO8601')
rheas_chirps_raw['harvest']=pd.to_datetime(rheas_chirps_raw['harvest'], format='ISO8601')
rheas_chirps_raw['harvest']=pd.to_datetime(rheas_chirps_raw['harvest'], format='ISO8601')
rheas_chirps_raw['year'] = rheas_chirps_raw['planting'].dt.year
rheas_chirps_raw['yield']=rheas_chirps_raw['gwad']/1000

In [16]:
# MAM and OND season OND =[10,11,12] MAM = [3,4,5]
# OND season

seasons = ['NONE', 'NONE', 'MAM', 'MAM','MAM', 'NONE', 'NONE', 'NONE', 'NONE', 'OND', 'OND', 'OND']
month_to_season=dict(zip(range(1,13), seasons))
rheas_tamsat_raw['season']= rheas_tamsat_raw['planting'].dt.month.map(month_to_season)
rheas_chirps_raw['season']= rheas_chirps_raw['planting'].dt.month.map(month_to_season)
rheas_chirps_raw[rheas_chirps_raw['season']=='NONE']

,id,gid,cname,season,ccode,ensemble,planting,harvest,wsgd,gwad,year,yield


In [42]:
# # Extract and aggregate only for MAM and OND
# extract data for 2023 only
rheas_tamsat_2023=rheas_tamsat_raw.loc[rheas_tamsat_raw['year']==2023]
rheas_tamsat_2022=rheas_tamsat_raw[rheas_tamsat_raw['year']==2022]
rheas_chirps_2023=rheas_chirps_raw[rheas_chirps_raw['year']==2023]
rheas_chirps_2022=rheas_chirps_raw[rheas_chirps_raw['year']==2022]

# extract for MAM and OND
# tamsat
rheas_tamsat_MAM_2022= rheas_tamsat_2022.loc[rheas_tamsat_2022['season']=='MAM']
rheas_tamsat_MAM_2023= rheas_tamsat_2023.loc[rheas_tamsat_2023['season']=='MAM']
rheas_tamsat_OND_2022= rheas_tamsat_2022.loc[rheas_tamsat_2022['season']=='OND']

# chirps
rheas_chirps_MAM_2022= rheas_chirps_2022.loc[rheas_chirps_2022['season']=='MAM']
rheas_chirps_MAM_2023= rheas_chirps_2023.loc[rheas_chirps_2023['season']=='MAM']
rheas_chirps_OND_2022= rheas_chirps_2022.loc[rheas_chirps_2022['season']=='OND']



# aggregate 
# tamsat
rheas_tamsat_MAM_2022=rheas_tamsat_MAM_2022.groupby(['cname'], as_index=False).agg({'yield': 'mean'})
rheas_tamsat_MAM_2023=rheas_tamsat_MAM_2023.groupby(['cname'], as_index=False).agg({'yield': 'mean'})
rheas_tamsat_OND_2022=rheas_tamsat_OND_2022.groupby(['cname'], as_index=False).agg({'yield': 'mean'})

# chirps
rheas_chirps_MAM_2022=rheas_chirps_MAM_2022.groupby(['cname'], as_index=False).agg({'yield': 'mean'})
rheas_chirps_MAM_2023=rheas_chirps_MAM_2023.groupby(['cname'], as_index=False).agg({'yield': 'mean'})
rheas_chirps_OND_2022=rheas_chirps_OND_2022.groupby(['cname'], as_index=False).agg({'yield': 'mean'})

In [43]:
# Read shapefiles and rename columns
country_shp = gpd.read_file("data/shp/gadm41_KEN_0.shp")
county_shp = gpd.read_file("data/shp/gadm41_KEN_1.shp")


# Rename kenya_shp 
county_shp=county_shp.rename(columns={'NAME1':'cname'})

In [44]:
# merge with shapefile
# tamsat
tamsat_mam_2022 = pd.merge(county_shp, rheas_tamsat_MAM_2022, on="cname")
tamsat_mam_2023= pd.merge(county_shp, rheas_tamsat_MAM_2023, on="cname")
tamsat_ond_2022= pd.merge(county_shp, rheas_tamsat_OND_2022, on="cname")


# chirps
chirps_mam_2022 = pd.merge(county_shp, rheas_chirps_MAM_2022, on="cname")
chirps_mam_2023= pd.merge(county_shp, rheas_chirps_MAM_2023, on="cname")
chirps_ond_2022= pd.merge(county_shp, rheas_chirps_OND_2022, on="cname")
# tamsat_mam_2022.columns

In [45]:
# convert to GeoJSONDataSource
# tamsat
country_geo_source = GeoJSONDataSource(geojson=country_shp.to_json())
geo_source_tamsat_mam_2022 = GeoJSONDataSource(geojson=tamsat_mam_2022.to_json())
geo_source_tamsat_mam_2023= GeoJSONDataSource(geojson=tamsat_mam_2023.to_json())
geo_source_tamsat_ond_2022= GeoJSONDataSource(geojson=tamsat_ond_2022.to_json())

# chirps
country_geo_source = GeoJSONDataSource(geojson=country_shp.to_json())
geo_source_chirps_mam_2022 = GeoJSONDataSource(geojson=chirps_mam_2022.to_json())
geo_source_chirps_mam_2023= GeoJSONDataSource(geojson=chirps_mam_2023.to_json())
geo_source_chirps_ond_2022= GeoJSONDataSource(geojson=chirps_ond_2022.to_json())

In [46]:
# MAM 2022
color_mapper = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
p1 = figure(title='MAM Maize Yield (mt/ha) - 2022',
           outer_width=650, outer_height=600)#Tamsat

p1.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p1.patches('xs','ys', source=geo_source_tamsat_mam_2022,
          fill_color={'field': 'yield', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('County: ', '@cname'), ('Yield', '@yield')])
p1.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p1.add_layout(color_bar, 'right')

In [47]:
# MAM 2022
color_mapper = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
p2 = figure(title='Chirps MAM Yield (mt/ha) - 2022',
           outer_width=650, outer_height=600)

p2.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p2.patches('xs','ys', source=geo_source_chirps_mam_2022,
          fill_color={'field': 'yield', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('County: ', '@cname'), ('Yield', '@yield')])
p2.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p2.add_layout(color_bar, 'right')

In [48]:
# MAM 2023
color_mapper = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
p3 = figure(title='MAM Maize Yield (mt/ha) - 2023',
           outer_width=650, outer_height=600)#Tamsat

p3.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p3.patches('xs','ys', source=geo_source_tamsat_mam_2023,
          fill_color={'field': 'yield', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('county: ', '@cname'), ('Yield', '@yield')])
p3.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p3.add_layout(color_bar, 'right')

In [49]:
# MAM 2023
color_mapper = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
p4 = figure(title='Chirps MAM Yield (mt/ha) - 2023',
           outer_width=650, outer_height=600)

p4.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p4.patches('xs','ys', source=geo_source_chirps_mam_2023,
          fill_color={'field': 'yield', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('county: ', '@cname'), ('Yield', '@yield')])
p4.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p4.add_layout(color_bar, 'right')

In [50]:
# Short Rains Season 2022
color_mapper1 = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
p5 = figure(title='OND Maize Yield(kg/ha) - 2022',
           outer_width=650, outer_height=600)#Tamsat 

p5.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p5.patches('xs','ys', source=geo_source_tamsat_ond_2022,
          fill_color={'field': 'yield', 'transform': color_mapper1},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('County', '@cname'), ('Yield', '@yield')])
p5.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p5.add_layout(color_bar, 'right')

In [51]:
# Short Rains Season 2022
color_mapper1 = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
p6 = figure(title='Chirps OND Yield(kg/ha) - 2022',
           outer_width=650, outer_height=600)

p6.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p6.patches('xs','ys', source=geo_source_chirps_ond_2022,
          fill_color={'field': 'yield', 'transform': color_mapper1},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('County', '@cname'), ('Yield', '@yield')])
p6.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p6.add_layout(color_bar, 'right')

In [52]:
# # ploting the MAM for 2022 and 2023
# rheas_tamsat_MAM_2022= rheas_tamsat_2022.loc[rheas_tamsat_2022['season']=='MAM']
# rheas_tamsat_MAM_2023= rheas_tamsat_2023.loc[rheas_tamsat_2023['season']=='MAM']
# rheas_tamsat_MAM_2022=rheas_tamsat_MAM_2022.groupby(['year', 'cname'], as_index=False).agg({'yield': 'mean'})
# rheas_tamsat_MAM_2023=rheas_tamsat_MAM_2023.groupby(['year', 'cname'], as_index=False).agg({'yield': 'mean'})

# # starting plotting
# import matplotlib.pyplot as plt

# X=list(rheas_tamsat_MAM_2022['cname'])
# year_2023=list(rheas_tamsat_MAM_2023['yield'])
# year_2022=list(rheas_tamsat_MAM_2022['yield'])
# X_axis = np.arange(len(X))
# plt.bar(X_axis - 0.2, year_2023, 0.4, label = 'yield_2023')
# # plt.bar(X_axis + 0.2, year_2022, 0.4, label = 'yield_2022')
# plt.xticks(X_axis)
# plt.xlabel("Groups")
# plt.ylabel("Number of Students")
# plt.title("Number of Students in each group")
# plt.legend()
# plt.show()

In [53]:

p=gridplot([p1, p3, p5], ncols=2, width=650, height=600)#Tamsat only
# p=gridplot([p2, p4, p6],ncols=3, width=650, height=600)#Chirps only
# p=gridplot([p1, p2, p3, p4, p5, p6],ncols=3, width=650, height=600)#Chirps and tamsat


show(p)

In [54]:
from bokeh.io import save
# # specify the output file name and location
save(p, filename='../kenya.html', title='Maize Yield Maps')

C:\Users\kasit\AppData\Local\Temp\ipykernel_1440\2676417183.py:3: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  save(p, filename='../kenya.html', title='Maize Yield Maps')


'd:\\RONO\\rheas\\maize_crop_modeling\\kenya.html'